In [25]:
# ! pip install mysql-connector-python
# conda install -c anaconda sqlalchemy
# !pip install recommenders
# conda install -c anaconda pymysql


import numpy as np
import pandas as pd
import mysql.connector
import sqlalchemy

from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import types
from sqlalchemy import create_engine
from pathlib import Path

In [26]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=""
)

sqlEngine= create_engine('mysql+pymysql://root:@localhost:3306/tutorcatdb')

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('epl_db',)
('information_schema',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('projectdb',)
('spl_db2',)
('testdb',)
('tutorcatdb',)


In [27]:
notes_Rating_DataFrame = pd.read_sql('SELECT NotesID,StudentID,Rating FROM notes_rating',con=sqlEngine)

notes_Rating_DataFrame

,NotesID,StudentID,Rating
0,25,10001,1
1,25,10002,4
2,25,10003,4
3,26,10001,3
4,26,10003,3
5,27,10003,4
6,28,10001,4
7,28,10002,4
8,28,10003,2
9,29,10002,5


In [28]:
print("Notes Number : " + str(notes_Rating_DataFrame['NotesID'].nunique()) )

Notes Number : 8


playHoursAvgDf : col( gameName	playHours)

In [29]:
piv_dataFrame=notes_Rating_DataFrame.pivot_table(index='StudentID', columns='NotesID',values="Rating").fillna(0)

In [30]:
piv_dataFrame

NotesID,25,26,27,28,29,30,31,32
StudentID,,,,,,,,
10001,1.0,3.0,0.0,4.0,0.0,5.0,0.0,1.0
10002,4.0,0.0,0.0,4.0,5.0,5.0,5.0,5.0
10003,4.0,3.0,4.0,2.0,5.0,3.0,4.0,0.0


In [31]:
def standardize(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row

piv_dataFrame = piv_dataFrame.apply(standardize)

In [32]:
piv_dataFrame

NotesID,25,26,27,28,29,30,31,32
StudentID,,,,,,,,
10001,-0.666667,0.333333,-0.333333,0.333333,-0.666667,0.333333,-0.6,-0.2
10002,0.333333,-0.666667,-0.333333,0.333333,0.333333,0.333333,0.4,0.6
10003,0.333333,0.333333,0.666667,-0.666667,0.333333,-0.666667,0.2,-0.4


In [33]:
piv_dataFrame = piv_dataFrame.fillna(0)
notes_similarity = cosine_similarity(piv_dataFrame.T)
print(len(notes_similarity))

8


In [34]:
notes_similarity_df = pd.DataFrame(notes_similarity,index=piv_dataFrame.columns,columns=piv_dataFrame.columns)

notes_similarity_df

NotesID,25,26,27,28,29,30,31,32
NotesID,,,,,,,,
25,1.000000,-0.500000,0.500000,-0.500000,1.000000,-0.500000,0.981981,0.327327
26,-0.500000,1.000000,0.500000,-0.500000,-0.500000,-0.500000,-0.654654,-0.981981
27,0.500000,0.500000,1.000000,-1.000000,0.500000,-1.000000,0.327327,-0.654654
28,-0.500000,-0.500000,-1.000000,1.000000,-0.500000,1.000000,-0.327327,0.654654
29,1.000000,-0.500000,0.500000,-0.500000,1.000000,-0.500000,0.981981,0.327327
30,-0.500000,-0.500000,-1.000000,1.000000,-0.500000,1.000000,-0.327327,0.654654
31,0.981981,-0.654654,0.327327,-0.327327,0.981981,-0.327327,1.000000,0.500000
32,0.327327,-0.981981,-0.654654,0.654654,0.327327,0.654654,0.500000,1.000000


In [35]:
# df = game_similarity_df.reset_index(index=game_similarity_df.loc[:,0])
df = notes_similarity_df.reset_index()
df = df.set_index('NotesID')
df = df.reset_index()

testFram = pd.melt(df,id_vars=["NotesID"],var_name="NotesID2",value_name='Val')
testFram

testFram.to_sql('ai_table', con=sqlEngine, if_exists='replace',index=False, dtype={"Val": types.Float()})
testFram

,NotesID,NotesID2,Val
0,25,25,1.000000
1,26,25,-0.500000
2,27,25,0.500000
3,28,25,-0.500000
4,29,25,1.000000
...,...,...,...
59,28,32,0.654654
60,29,32,0.327327
61,30,32,0.654654
62,31,32,0.500000


In [36]:
# def get_similarity_game(notesID,rating):
#
#     similarity_mark = game_similarity_df[game_name]*user_rating
#     similarity_mark = similarity_mark.sort_values(ascending=False)
#
#     return similarity_mark
#
# print(get_similarity_game("Warframe",5))

